In [1]:
include("TidalLoveNumbersPorousK23.jl")
using .TidalLoveNumbers
using DoubleFloats
using PyPlot
using Statistics
using PyCall
using BenchmarkTools

prec = TidalLoveNumbers.prec
precc = TidalLoveNumbers.precc

Complex{Double64}

In [2]:
non_dim = false

G = prec(6.6743e-11)
e = 0.0041

h_core = 700.0 - 690.0
h_mantle_low = 800. + 690
h_mantle_up = 300. - 0.

h_crust = 20.
ω0 = 2*2.047e-5
ω = ω0

#enceladus test model:
n = 2
ρₛ = [3300, 3300, 3300, prec(3300)]
r = [0, 
     h_core, 
     h_core+h_mantle_low, 
     h_core+h_mantle_low+h_mantle_up, 
     h_core+h_mantle_low+h_mantle_up+h_crust] .* 1e3
μ = [60+0im, 60, 60, prec(60)] .* 1e9
κ = [100e9, 100e9, 100e9, 100e9] .* 100000.0
η = [1e25, 1e25, 1e25, 1e25]

R = r[end]
μ0 = μ[end]
bp = 3
tp = 3


ρₗ = [0, 0, 3300, 0]
# α  = [0, 0, 0.95, 0., 0]
κₗ = [0, 0, 100e9, 0] .* 100000.0
κd = 0.05κ
α = zero(κd)
α[3] = κd[3]/κ[3]
k = [0, 0, 1e-7, 0]

ηₗ = [0, 0, 1.0, 0]
ϕ =  [0, 0, prec(0.1), 0]

ρ = (1 .- ϕ) .* ρₛ + ϕ .* ρₗ # bulk density
# ρ = ρₛ
r = expand_layers(r)
g = get_g(r, ρ)


301×4 Matrix{Double64}:
 0.0                     0.009225890013055914  …  1.6606602023500647
 3.075296671018639e-5    0.013808082052873687     1.660721708283485
 6.150593342037278e-5    0.018390274092691457     1.6607832142169054
 9.225890013055916e-5    0.02297246613250923      1.6608447201503258
 0.00012301186684074555  0.027554658172327        1.6609062260837464
 0.00015376483355093192  0.03213685021214477   …  1.6609677320171665
 0.0001845178002611183   0.03671904225196254      1.6610292379505869
 0.0002152707669713047   0.041301234291780314     1.6610907438840075
 0.0002460237336814911   0.04588342633159809      1.6611522498174276
 0.00027677670039167744  0.05046561837141585      1.6612137557508482
 ⋮                                             ⋱  
 0.008979866279374424    1.347225965639845        1.6786199349088136
 0.00901061924608461     1.3518081576796628       1.678681440842234
 0.009041372212794796    1.3563903497194807       1.6787429467756543
 0.009072125179504982    1.360

In [3]:
B = zeros(precc, 8, 8)

8×8 Matrix{Complex{Double64}}:
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  …  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im     0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im     0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im     0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im     0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  …  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im     0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im     0.0 + 0.0im  0.0 + 0.0im

In [4]:
test1() = for i in 1:1000 get_B!(B, r[1,3], r[2,3], g[2,3], g[1,3], ρ[3], μ[3], κ[3], ω, ρₗ[3], κₗ[3], κd[3], α[3], ηₗ[3], ϕ[3], k[3]) end

test1 (generic function with 1 method)

In [11]:
@btime test1()

  83.308 ms (711000 allocations: 105.59 MiB)


In [5]:
@btime test1()

  83.720 ms (711000 allocations: 105.59 MiB)


In [3]:
res = 10.0
lons = deg2rad.(collect(0:res:360-0.001))'
clats = deg2rad.(collect(0:res:180))

lons_deg = rad2deg.(lons[:])
lats_deg = 90 .- rad2deg.(clats)
dres = deg2rad(res);

# Bulk dissipation

In [4]:
fig, axes = plt.subplots(ncols=2, figsize=(10,4))

(Figure(PyObject <Figure size 1000x400 with 2 Axes>), PyObject[PyObject <Axes: >, PyObject <Axes: >])

In [5]:
# κ = [100e9, 100e9, 10e9 + 0im, 100e9] 
# η = [1e30, 1e30, 1e20, 1e30]
# η_solid = [1e30, 1e30, 1e20, 1e30]
# μ = [60+0im, 60, 60, prec(60)] .* 1e9

# ρₗ = [0, 0, 3300, 0]
# # α  = [0, 0, 0.95, 0., 0]
# κₗ = [0, 0, 1e9, 0] 
# κd = copy(κ)
# κd[3] = 0.05 * κ[3]

# # Set Biot's coefficient
# α = zero(κd)
# α[3] = κd[3]/κ[3]

# # α = [0, 0, 0.0+0im, 0.0]
# k = [0, 0, 1e-8, 0]

# ηₗ = [0, 0, 1e-2, 0]

# ϕ =  [0, 0, prec(0.3), 0]

# ρ = (1 .- ϕ) .* ρₛ + ϕ .* ρₗ # bulk density

# g = get_g(r, ρ)



# ηs_shear = 20:1:21
# ηs_bulk = 15:1:20
# ks = -9:1:-6

# Edot = zeros((length(ηs_shear), length(ηs_bulk)))
# Edots = zeros((length(ηs_shear), length(ηs_bulk)))
# k_ln = zeros(ComplexF64, (length(ηs_shear), length(ηs_bulk)))
# k_lns = zeros(ComplexF64, (length(ηs_shear), length(ηs_bulk)))

# # println(size(axes))

# colors = ["C1", "C2", "C3", "C4"]
# cmap = PyPlot.cm.cool
# colors = [0.2, 0.4, 0.6, 0.8]
# cnt = 0
# for j in eachindex(ηs_bulk)
#     for i in eachindex(ηs_shear)
#         #######################################################################
#         # η[3] = 10.0^ηs[i] * (non_dim ? 1.0/(μ0*T) : 1.0)
        
#         κdc = copy(κd)
#         κdc = 1im * ω * κd ./ (1im * ω .+ κd ./ (10.0^ηs_bulk[j]))

#         # println(κdc)

#         αc = copy(α)
#         αc[3] = κdc[3] / κd[3]

#         # println(αc)

#         # κd[3] = κd
#         # κdc[3] = 1.0 / (1 / (κ[3]*(1 - 0.95)) + 1 / (1im * ω * η[3]))
        
#         # κs = copy(κ)
#         # κs[3] = κd[3]/(1-0.95)
#         # κd .= 0.05κ

#         # k = 10^-j * eta_l 
#         # k[3] = 10.0^ks[j] * (non_dim ? 1.0/R^2 : 1.0)

#         μc =  1im*ω*μ ./ (1im*ω .+ μ ./ (10.0 .^ ηs_shear[i]))


#         μc = μ

#         y = ComplexF64.(calculate_y(r, ρ, g, μc, κ, ω, ρₗ, κₗ, κd, αc, ηₗ, ϕ, k))

        
#         κs = copy(κ)
#         κs[3] = κdc[3]

#         y2 = ComplexF64.(calculate_y(r, ρ, g, μc, κs))

#         yR = y[:,end,end]
#         yR2 = y2[:,end,end]

#         k2 = yR[5] - 1
#         k22 = yR2[5] - 1

#         k_ln[i,j] = k2
#         k_lns[i,j] = k22
    
#         Edot[i,j] = 21/2 * -imag(k2) * (ω0*R)^5/G * e^2
#         Edots[i] = 21/2 * -imag(k22) * (ω0*R)^5/G * e^2

       

#     end
#     cnt += 1
# end

# # c = axes[2].contourf(10.0 .^ collect(ηs_shear), 10.0 .^ collect(ηs_bulk), Edot' ./ 1e12)
# # cbar = plt.colorbar(c)
# # # end
# axes[2].loglog(10.0 .^ collect(ηs_bulk), abs.(Edot[1,:])/1e12, lw=3.0)

# axes[2].set_xscale("log")
# axes[2].set_yscale("log")

# # axes[2].legend(frameon=false)

# # axes[1].set_ylabel("Tidal Love Number \$|k_2|\$")
# # axes[2].set_ylabel("Dissipated Power [TW]")
# # axes[1].set_xlabel("Asthenosphere Viscosity [Pa s]")
# # axes[2].set_xlabel("Asthenosphere Viscosity [Pa s]")

# # axes[2].axhspan((9.33-1.87)*1e13/1e12, (9.33+1.87)*1e13/1e12, alpha=0.8)
# # axes[1].set_ylim([0,1.3])
# # fig.savefig("edot_-6.pdf", bbox_inches="tight", transparent=true, dpi=350)
# display(fig)
# axes[1].clear()
# axes[2].clear()
# # cbar.remove()

In [6]:
using ProfileCanvas

In [7]:
for i in 1:3
    @btime y = ComplexF64.(calculate_y(r, ρ, g, μ, κ, ω, ρₗ, κₗ, κd, α, ηₗ, ϕ, k))
end

  71.781 ms (403263 allocations: 126.99 MiB)
  72.764 ms (403263 allocations: 126.99 MiB)
  72.297 ms (403263 allocations: 126.99 MiB)


In [9]:
ProfileCanvas.@profview y = ComplexF64.(calculate_y(r, ρ, g, μ, κ, ω, ρₗ, κₗ, κd, α, ηₗ, ϕ, k))